
# Analysis Comparison between Original and Expanded Lexicons on Cohort Analysis Task

Subreddit: bipolar2

In [1]:
from pathlib import Path
import pandas as pd

from config import global_config
from lexicons2 import Values, ValuesExpanded, Liwc2015, Liwc2015Expanded
from spaces import WordEmbeddings, LabelEmbeddings

In [2]:
COHORT_NAME = 'bipolar2'

In [3]:
cohort_corpus_path = global_config.reddit.submissions[COHORT_NAME]
cohort_models_path = f'{global_config.paths.models}/{COHORT_NAME}'

control_corpus_path = global_config.reddit.submissions['IAMA']
control_models_path = f'{global_config.paths.models}/IAMA'

In [4]:
cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks.csv'
expanded_cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks_expanded.csv'

control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks.csv'
expanded_control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks_expanded.csv'

relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks.csv'
expanded_relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks_expanded.csv'

Build Control Spaces

In [5]:
control_word_space = WordEmbeddings(corpus_path=control_corpus_path, model_path=control_models_path).build()

In [6]:
org_control_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=control_word_space).build()

In [7]:
expanded_control_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=control_word_space).build()

Build Cohort Spaces

In [8]:
cohort_word_space = WordEmbeddings(corpus_path=cohort_corpus_path, model_path=cohort_models_path).build()

In [9]:
org_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=cohort_word_space).build()

In [10]:
expanded_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=cohort_word_space).build()

Save and view results

In [11]:
def save_if_not_save_and_return(space: LabelEmbeddings, path: str):
	if not Path(path).exists():
		space.save_distances_to_csv(path)
	return pd.read_csv(path, names=['label_one', 'label_two', 'distance'])

control_ranks = save_if_not_save_and_return(org_control_label_space, control_ranks_csv)
expanded_control_ranks = save_if_not_save_and_return(expanded_control_label_space, expanded_control_ranks_csv)

cohort_ranks = save_if_not_save_and_return(org_cohort_label_space, cohort_ranks_csv)
expanded_cohort_ranks = save_if_not_save_and_return(expanded_cohort_label_space, expanded_cohort_ranks_csv)

In [12]:
org_cohort_label_space.compute_rank_deltas(org_control_label_space, relative_ranks_csv)
relative_ranks = pd.read_csv(relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

expanded_cohort_label_space.compute_rank_deltas(expanded_control_label_space, expanded_relative_ranks_csv)
expanded_relative_ranks = pd.read_csv(expanded_relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

View Stuff

In [13]:
relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,values:feeling-good,69,43,0.195169,0.499940,26,-0.304771
1,liwc2015:function,values:learning,51,28,0.029772,0.178845,23,-0.149072
2,liwc2015:function,values:truth,77,55,0.478888,0.713501,22,-0.234613
3,liwc2015:function,liwc2015:we,40,22,0.011880,0.130866,18,-0.118986
4,liwc2015:function,values:animals,82,64,1.342300,0.794112,18,0.548188
...,...,...,...,...,...,...,...,...
6967,values:animals,liwc2015:sexual,11,75,1.228608,1.021016,-64,0.207592
6968,values:animals,values:children,9,76,1.194637,1.028190,-67,0.166448
6969,values:animals,liwc2015:nonflu,6,74,1.175533,1.009550,-68,0.165982
6970,values:animals,liwc2015:filler,0,69,1.074962,0.980616,-69,0.094346


In [14]:
expanded_relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,liwc2015:we,45,25,0.011880,0.130866,20,-0.118986
1,liwc2015:function,liwc2015:work,57,39,0.025867,0.336770,18,-0.310903
2,liwc2015:function,liwc2015:money,73,58,0.177593,0.681783,15,-0.504190
3,liwc2015:function,liwc2015:pconcern,67,54,0.071297,0.624154,13,-0.552857
4,liwc2015:function,liwc2015:anger,74,61,0.190324,0.719446,13,-0.529122
...,...,...,...,...,...,...,...,...
7135,liwc2015:filler,values:order,15,50,0.589706,0.989284,-35,-0.399577
7136,liwc2015:filler,liwc2015:affiliation,9,47,0.586435,0.985804,-38,-0.399368
7137,liwc2015:filler,liwc2015:certain,10,65,0.586919,1.022467,-55,-0.435548
7138,liwc2015:filler,liwc2015:affect,3,58,0.580702,1.000825,-55,-0.420123
